In [1]:
print("dsadsadsa")

dsadsadsa


In [4]:
import pandas as pd
from sklearn.utils import resample
from pathlib import Path

dataset_dir = Path("./dataset")
dataset_dir.mkdir(parents=True, exist_ok=True)  # Ensure the directory exists
dataset_path = dataset_dir / "creditcard-data.csv"
df = pd.read_csv(dataset_path)
df.head

<bound method NDFrame.head of             Time         V1         V2        V3        V4        V5  \
0            0.0  -1.359807  -0.072781  2.536347  1.378155 -0.338321   
1            0.0   1.191857   0.266151  0.166480  0.448154  0.060018   
2            1.0  -1.358354  -1.340163  1.773209  0.379780 -0.503198   
3            1.0  -0.966272  -0.185226  1.792993 -0.863291 -0.010309   
4            2.0  -1.158233   0.877737  1.548718  0.403034 -0.407193   
...          ...        ...        ...       ...       ...       ...   
284802  172786.0 -11.881118  10.071785 -9.834783 -2.066656 -5.364473   
284803  172787.0  -0.732789  -0.055080  2.035030 -0.738589  0.868229   
284804  172788.0   1.919565  -0.301254 -3.249640 -0.557828  2.630515   
284805  172788.0  -0.240440   0.530483  0.702510  0.689799 -0.377961   
284806  172792.0  -0.533413  -0.189733  0.703337 -0.506271 -0.012546   

              V6        V7        V8        V9  ...       V21       V22  \
0       0.462388  0.239599  0.

In [5]:
# Load the dataset (Ensure you have 'creditcard.csv' in your working directory)
df = pd.read_csv(dataset_path)

# Check class distribution
print("Original class distribution:")
print(df["Class"].value_counts())

# Separate majority and minority classes
df_majority = df[df["Class"] == 0]  # Non-fraudulent transactions
df_minority = df[df["Class"] == 1]  # Fraudulent transactions

# Apply undersampling: Reduce majority class to match minority class size
df_majority_undersampled = resample(df_majority, 
                                    replace=False,    # Sample without replacement
                                    n_samples=len(df_minority),  # Match minority class size
                                    random_state=42)  # Reproducibility

# Combine minority class with undersampled majority class
df_undersampled = pd.concat([df_majority_undersampled, df_minority])

# Check new class distribution
print("\nNew class distribution after undersampling:")
print(df_undersampled["Class"].value_counts())

# Save the new dataset
new_dataset_path = dataset_dir / "creditcard_undersampled.csv"
df_undersampled.to_csv(new_dataset_path, index=False)
print("\nUndersampled dataset saved as 'creditcard_undersampled.csv'.")

Original class distribution:
Class
0    284315
1       492
Name: count, dtype: int64

New class distribution after undersampling:
Class
0    492
1    492
Name: count, dtype: int64

Undersampled dataset saved as 'creditcard_undersampled.csv'.


In [17]:
from minio import Minio

client = Minio("192.168.2.65:9001",
    access_key="useraccesskey",
    secret_key="useraccesskey",
)


In [18]:
source_file =new_dataset_path
bucket_name = "dataset-bucket"
found = client.bucket_exists(bucket_name)
if not found:
    client.make_bucket(bucket_name)
    print("Created bucket", bucket_name)
else:
    print("Bucket", bucket_name, "already exists")



MaxRetryError: HTTPSConnectionPool(host='192.168.2.65', port=9001): Max retries exceeded with url: /dataset-bucket?location= (Caused by SSLError(SSLError(1, '[SSL: WRONG_VERSION_NUMBER] wrong version number (_ssl.c:1028)')))

In [ ]:
import boto3

# MinIO credentials and endpoint
endpoint_url = "http://localhost:9000"  # Adjust if running remotely
aws_access_key_id = "user"
aws_secret_access_key = "WW4e69Wwcv0w"
bucket_name = "models-bucket"
file_path = "userprefix.json"
object_name = "uploaded_example.txt"

# Create S3 client
s3_client = boto3.client(
    "s3",
    endpoint_url=endpoint_url,  # MinIO URL
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key
)

# Upload file
s3_client.upload_file(file_path, bucket_name, object_name)

print(f"File {file_path} uploaded to {bucket_name} as {object_name}")